# RNN으로 이미지 분류하기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(60000, 28, 28) (10000, 28, 28)
(60000,) (10000,)


In [6]:
y_train[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

## 정규화

In [7]:
X_train = X_train.astype('float32')/255.
X_test = X_test.astype('float32')/255.

In [8]:
# 원핫 인코딩 하지않고 그대로 넣을 때에는 compile 시에 다르게 loss를 넣음
from tensorflow.keras.utils import to_categorical

y_trian_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)

y_trian_oh.shape

(60000, 10)

In [9]:
y_trian_oh[:5]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

## 모델

In [10]:
X_train.shape
# CNN 레이어에 넣기 위해서는 reshape(-1, 28, 28, 1) 해야했음

(60000, 28, 28)

In [11]:
model = keras.Sequential([
    layers.LSTM(64, activation='tanh', input_shape=(28, 28)),
    # 한번에 28개씩(1줄), 28번(time step, 줄개수) 넣음
    
    layers.Dense(10, activation='softmax')
])

In [12]:
model.compile(
    optimizer = 'rmsprop',
    loss = 'categorical_crossentropy',
    metrics = ['acc']
)

In [15]:
EPOCHS = 30
BATCH_SIZE = 32

log = model.fit(X_train, y_trian_oh,
                epochs = EPOCHS,
                batch_size = BATCH_SIZE,
                validation_split = 0.2,
                verbose = 1)

Epoch 1/30
1500/1500 [==============================] - 14s 5ms/step - loss: 0.5441 - acc: 0.8253 - val_loss: 0.2286 - val_acc: 0.9302
Epoch 2/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.1787 - acc: 0.9468 - val_loss: 0.1290 - val_acc: 0.9613
Epoch 3/30
1500/1500 [==============================] - 7s 5ms/step - loss: 0.1197 - acc: 0.9645 - val_loss: 0.1129 - val_acc: 0.9678
Epoch 4/30
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0924 - acc: 0.9725 - val_loss: 0.0841 - val_acc: 0.9761
Epoch 5/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0751 - acc: 0.9774 - val_loss: 0.0772 - val_acc: 0.9771
Epoch 6/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0622 - acc: 0.9813 - val_loss: 0.0761 - val_acc: 0.9781
Epoch 7/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0546 - acc: 0.9828 - val_loss: 0.0736 - val_acc: 0.9796
Epoch 8/30
1500/1500 [==============================] - 7s 5m

In [16]:
model.evaluate(X_test, y_test_oh)

313/313 [==============================] - 1s 3ms/step - loss: 0.0915 - acc: 0.9825


[0.09150798618793488, 0.9825000166893005]

In [17]:
model.save('my_rnn_model.h5')

In [18]:
model2 = tf.keras.models.load_model('my_rnn_model.h5')

In [19]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                23808     
                                                                 
 dense (Dense)               (None, 10)                650       
                                                                 
Total params: 24,458
Trainable params: 24,458
Non-trainable params: 0
_________________________________________________________________


In [20]:
model2.evaluate(X_test, y_test_oh)

313/313 [==============================] - 1s 3ms/step - loss: 0.0915 - acc: 0.9825


[0.09150798618793488, 0.9825000166893005]

In [ ]:
# 형식 저장 안할 경우 파일이 여럿 생김, pb = 텐서플로우 형식, 레이어 정보 등이 들어있다

In [21]:
# 바로 구글 드라이브에 모델을 저장하고 싶을 경우,
# 구글 드라이브에 마운트 한 뒤에 (드라이브 아이콘 클릭)
model.save('/content/drive/MyDrive/model/rnn_mnist.h5')